# ID2221: Data Intensive Computing
# Lab 2 - Spark and Spark SQL
## (Updated 2017-09-23)

In this part of the lab you will practice the basic operations of Spark (RDDs) and Spark SQL (DataFrames). Next you will use what you learned to do some interactive spark analytics.


## The entry point: SparkSession
This is the main entry point to all Spark functionality

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row

In [ ]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Spark Basics') \
    .getOrCreate()

We can check which version of spark we are using

In [ ]:
spark.version

To access the RDD api we can get the SparkContext form our SparkSession

In [ ]:
sc = spark.sparkContext

## Basic Operations

## Create an RDD from Python list

We start by createing a "local" python list, in this example `myList`. This list is a normal Python collection stored in the memory of your machine, not in Spark yet

In [ ]:
myList = [1, 2, 3, 4, 5, 6]

In [ ]:
myList

Then, we can create an RDD from a Python collection using `parallelize()`. This will divide the collection into a number of partitions, distribute it on the servers in your Spark cluster, and return you a reference to it (`nums` in this example) which is an RDD Resilient Distributed Dataset. 

In [ ]:
nums = sc.parallelize(myList)

In [ ]:
nums

Optionally, you can manually set the number of partitions as parameter to `sc.parallelize(myList, 2)`. You can also repartition an RDD with `nums2 = nums.repartition(2)`, and check the number of partitions with `nums.getNumPartitions()`

In [ ]:
nums.getNumPartitions()

You can get help on any Python function using `help()`

In [ ]:
help(nums.getNumPartitions)

You can execute linux shell commands by prepending an exclamation mark (!) to a shell command

In [ ]:
!ls

## Some Actions

Remember, actions trigger computations and produce output (e.g., print on your screen). Other transformations are "lazy"

You can convert an RDD back to a python collection (e.g., to print it) using `collect()`. If it is too large you can get first few elements with `take()` or a random sample with `takeSample()`.

In [ ]:
nums.collect()

In [ ]:
nums.take(2)

In [ ]:
nums.takeSample(False, 2)

Count the number of elements

In [ ]:
nums.count()

## Map
Apply a function to all elements of an RDD

In [ ]:
squares = nums.map(lambda x: x**2)

In [ ]:
squares.collect()

Filter using a boolean function

In [ ]:
even = nums.filter(lambda x: x%2 == 0)

In [ ]:
even.collect()

## Map vs. FlatMap
FlatMap generates zero or more elements for each input

In [ ]:
many_nums = nums.flatMap(lambda x: list(range(0, x)))

In [ ]:
many_nums.collect()

Map is one to one

In [ ]:
lists = nums.map(lambda x: list(range(0, x)))

In [ ]:
lists.collect()

## Key-Value Pairs

RDDs support simple data structure key-value in the from (k, v). For example:

In [ ]:
users = sc.parallelize([('A', 20), ('B', 30), ('C', 40)])  # (Name, Age)

In [ ]:
users.keys().collect()

In [ ]:
users.values().collect()

In [ ]:
users.map(lambda x : (x[0]+'_wiser', x[1]+1)).collect()  # Assuming that you get wiser when you get older :D

In [ ]:
# apply map to values only
users.mapValues(lambda x : x**2).collect()

In [ ]:
users.collectAsMap()

## Reduce

In [ ]:
pets = sc.parallelize([('cat',1), ('dog',2), ('cat',3)])

In [ ]:
pets.collect()

In [ ]:
pets.lookup('cat')

In [ ]:
pets.groupByKey().mapValues(list).collect()

In [ ]:
pets.reduceByKey(lambda x, y: x+y).collect()

In [ ]:
from operator import add
pets.reduceByKey(add).collect()

## Join

In [ ]:
visits = sc.parallelize([('index.html','1.2.3.4'), ('about.html','3.2.3.4'), ('index.html','5.4.3.2'), ('help.html','7.6.1.2')])

In [ ]:
visits.collect()

In [ ]:
pageName = sc.parallelize([('index.html','Home'), ('about.html','About'), ('prod.html','Products')])

In [ ]:
visits.join(pageName).collect()

### Save

If your RDD has multiple partitions, the outpou will be split into an equal number of files. This allows writing in parallel into a distributed fule system and to keep the data distributed in a cluster.

First delete it if already exists

In [ ]:
! rm -rf nums.txt

In [ ]:
nums.saveAsTextFile('nums.txt')

# Word Count

In [ ]:
lines = sc.textFile('data/shakespeare.txt')

In [ ]:
lines.count()

In [ ]:
counts = lines.flatMap(lambda x: [y.strip('.,;:?!-') for y in x.split()]) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x,y:x+y)

In [ ]:
output = counts.collect()

In [ ]:
counts.count()

That is a lot of words to print on the screen! Lets take a sample to view

In [ ]:
counts.takeSample(False, 10)

What about the top used words? We can use `top()` for that. `top()` takes an optional key function that can be used to define the key used for sorting.

In [ ]:
counts.top(20, key=lambda x : x[1])  # Sort using the word count

What about top words having 5 or more characters?

We can modify the key function to return 0 if the word length is less than 5, otherwise, return the word count

In [ ]:
counts.top(20, key=lambda x : 0 if len(x[0]) < 5 else x[1])

# Spark SQL

Try the examples we discussed on the slides during the lecture

In [ ]:
l = [('Alice', 1)]
kids = spark.createDataFrame(l, ['name', 'age'])

In [ ]:
kids.show()

In [ ]:
kids.printSchema()

In [ ]:
schema = StructType( \
                    [StructField('name', StringType(), True), \
                     StructField('age', IntegerType(), True)])
users2 = spark.createDataFrame(users, schema)

In [ ]:
users2.show()

In [ ]:
users2.printSchema()

In [ ]:
df = spark.read.json('data/people.json')

In [ ]:
df.show()

In [ ]:
df.select('name').show()

In [ ]:
df.select(df['name'], df['age'] + 1).show()

In [ ]:
df.filter(df['age'] > 21).show()

In [ ]:
df.groupBy('age').count().show()

### Interoperating with RDDs

In [ ]:
# Load a text file and convert each line to a Row.
lines = sc.textFile('data/people.txt')
parts = lines.map(lambda l: l.split(','))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView('people')

In [ ]:
# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql('SELECT name FROM people WHERE age >= 13 AND age <= 19')

teenagers.show()

In [ ]:
# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teenNames = teenagers.rdd.map(lambda p: 'Name: ' + p.name).collect()
for name in teenNames:
    print(name)